<a href="https://colab.research.google.com/github/valeriia-nekrasova/compling/blob/main/%D0%A2%D1%80%D0%B5%D1%82%D1%8C%D0%B5_%D0%B4%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Коспект-шпаргалка по SMT**




# **Общая информация**

*Что такое SMT?*

*   Это разновидность машинного перевода, которая основа на поиске наиболее правдободобного перервода для языка на основе параллельного корпуса предложений исходного языка и языка, на который переводится предложение.
*   Наиболее точным переводом считается тот, при котором максимизируется вероятность употребления перевода слов в языке У при переводе с языка X (предложение на языке Х надо перевести на язык У)
*   Для начала строится языковая модель вероятности для У
*   Затем вычисляются допустимые пары Х-У с помощью статистических совпадений выравненных фраз в параллельном корпусе, который построен на вероятность перевода в языке Х при условии языка У
*   Далее максимизируется результат вычисления вероятность для языка У и корпусе, который построен на вероятность перевода в языке Х при условии языка У


**Компоненты SMT**

*N-граммная языковая модель* - моделируем вероятностное распределение конструкций на уровне слов или фраз в языке Y

*Модель перевода (t-model*) - собираем статистику соотвествий фраз в паралелльном корпусе, ищем переводческие соответствия X - Y и моделируем их с помощью теории вероятности:

Допускаем, что любое предложение языка Y может быть "искаженной" версией некой фразы на языке X

Ищем наиболее правдоподобные соответствия X - Y

*Декодер* - ищем наиболее грамматичные и лексически правдоподобные результаты, отбираем среди гипотез один результат

# **Первый этап - препроцессинг**

**Перед началом работы необходимо загрузить библиотеку  https://scikit-learn.org/stable/ **
Источник данных, на которых можно обучать модели для машинного перевода: OPUS Corpora. Это открытая коллекция параллельных корпусов текстов на различных языках: книги, статьи, переводы и другие типы текстов, которые могут быть использованы для обучения и оценки статистических моделей машинного перевода.
- На данном этапе необходимо выполнить загрузку данных:
извлечь данные из архива ( использовать модуль Python `tarfile` https://docs.python.org/3/library/tarfile.html ). Посмотреть список распакованных файлов можно с помощью shell команды ls
```
with tarfile.open('НАЗВАНИЕ_АРХИВА', 'r:ТИП_АРХИВА') as tar:
  tar.extractall()
```
- На основании извлеченных данных создаем выборки. Так делаем для каждого языка
```
with open('НАЗВАНИЕ_РАСПАКОВАННОГО_ФАЙЛА', 'r') as f:
  text = f.read().split('\n')[:-1]
```
- разделяем полученные выборки на обучающую и на тестовую с помощью функции `train_test_split` библиотеки `sklearn`. `array1` и `array2` - созданные ранее выборки
```
X_train, X_test, y_train, y_test = train_test_split(array1, array2)
```
На данном этапе также можно провести необходимую подготовку данных:
*   Отчистить данные
*   Выполнить токенизацию ( разделить каждое предложение по словам )
```
def tokenize(sentences):
  return [sentence.split() for sentence in sentences]
```
*   Также можно создать словарь для каждого языка: объединить слова со всех предложений



# **Второй этап - модель SMT**




## **1. IBM 1 Expectation-Maximization (t-model)**





Для начала необходимо создать t-table - таблицу вероятности соответствия слов. На первом шаге заполняем таблицу для каждого слова обучающей выборки значением вероятности того, что случайное слово x_vocab соответсвует случайному слову y_vocab, равным:

```
uniform = 1 / (len(x_vocab) * len(y_vocab))
```
После этого задаем кол-во итераций обучений и запускаем цикл по количеству итераций

В цикле необходимо выполнить 3 шага, каждый шаг - проход по обучающей выбоки в цикле

```
 for i in range(len(X_train)):
```
- Шаг 0:
  - создать слоты для подсчета статистики. Р(х|у) - условная вероятность совпадения в корпусе и Р(у) - статистическая языковая модель y.
- Шаг 1:
  - Необходимо посчитать статистику х. Для этого создаем  `total_stat = {}` и   
   собираем предварительную статистику на основе данных x : увеличиваем значение статистики при обноружении совместной встречаемости. По факту проходим по каждому слову и накапливаем сумму `t[(word_x, word_y)]`
  - После этого необходимо обновить данные для P(x|y) и P(y). Подсчитываем условную вероятность совпадений в корпусе (равномерное распределение / частотность x) и статистическую информации y (равномерное распределение / частотность x).
```
for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]
```
- Шаг 2: обновить t-table - для каждой пары х и у считаем величину: вероятность совпадения в корпусе / вероятность информации y
```
for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]
```

## **2. Биграммная модель**

Прежде всего объединяем две выборки для обучения модели и создаем хранилище для биграмм
```
tokens = ' '.join(german).split()
bigram_model = defaultdict(list)
```
Затем собираем все попарные совпадения: сохраняем в хранилище биграмм (слово, следующее слово)
```
for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)
```
Затем необходимо задать количество шагов, по аналогии, как мы делали в t-table.
Случайным образом выбираем токен с помощью модуля Python random и добавляем его в генерируемое предложение.
```
current_word = random.choice(tokens)
generated_sentence = current_word
```
Затем на каждом шаге выбираем правдободобные варианты продолжения для текущего токена и опять случайным образом выбираем новый токен и добавляем его в генерируемое предложение.
```
next_word_options = model[current_word]
current_word = random.choice(next_word_options)
generated_sentence += ' '
generated_sentence += current_word
```
В результате получаем сгенерированное заданным количеством шагов предложение


# **Третий этап - оценка результатов**

Оценивать результаты будем с помощью модуля Python NLTK - https://www.nltk.org/api/nltk.translate.bleu_score.html

Отсортируем `t-table` по убыванию правдоподобия, используя функцию `sorted` и установив `reverse = True`:
`sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)`
Реализуем функцию поиска наиболее подходящего токена:
```
def translate(token):
  for element in sorted_t:
    if element[0][1] == token:
      return element[0][0]
```
Теперь для каждого предложения из тестовой выборки получим его перевод: для каждого токена (слова)  из тестового предложения найдем наиболее подходящий перевод и запишем его в `translation`:
```
for sentence in y_test_tokens:
  translation = []
  for token in sentence:
    translation.append(translate(token))
```
Наконец, с помощью функции `corpus_bleu` из `nltk.translate.bleu_score`  сравним что выдает наша модель на тестовой выборке и правильный перевод из тестовой выборки. В результате получим `bleu_score`: чем он выше, тем лучше работает модель.
```
reference = [X_test_tokens[0], X_test_tokens[1]]
candidate = [translate(token) for token in y_test_tokens[0]]
bleu_score = corpus_bleu(reference, candidate)
```